## Load in the necessary libraries


In [337]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from tqdm import tqdm





## Load our first page

In [338]:
# load the webpage content

r = requests.get('https://www.vestibularfatec.com.br/classificacao/fatec.asp')

#convert to a beautiful soup object

soup = bs(r.content)

#print(soup.prettify())

#fatecs = soup.find_all("CodFatec option")
fatecs = soup.select("#CodFatec option")
#removi o option vazio
fatecs.pop(0)
#print((ftecs))

<option value="">Selecione...</option>

In [339]:
dfFatec = pd.DataFrame()
#dfFatec.columns = ['id','fatec']
objFatec = {} #construí um dicionário
listaFatecs = [] #construí uma lista
#dfFatec.head()

for fatec in fatecs:
    key = int(fatec['value'])
    value= fatec.get_text()
    objFatec[key] = str(value)
    listaFatecs.append({'id':key,'fatec':value})
    #print(fatec.string)
    #print(fatec.get_text())
    #print(fatec['value'])



In [340]:
#Criei uma DataFrame baseado na lista recém criada
dfFatec = pd.DataFrame(listaFatecs)

In [341]:
dfFatec.head()

,id,fatec
0,71,Adamantina - Fatec Adamantina
1,3,Americana - Fatec Americana - Ministro Ralph B...
2,35,Araçatuba - Fatec Araçatuba - Prof. Fernando A...
3,67,Araraquara - Fatec Araraquara
4,68,Araras - Fatec Araras


In [342]:
#salvo essa lista
dfFatec.to_csv('ds/listaFatec.csv', sep=';',index=False)


In [343]:
dfCursos = pd.DataFrame()
rCurso = requests.get('https://www.vestibularfatec.com.br/unidades-cursos/?q=')
#convert to a beautiful soup object
soupCurso = bs(rCurso.content)    
divCursos = soupCurso.find(id = "cursos")

listaCursos=divCursos.find_all("a")
listaCursosFatec = [{'id':245,'curso':'Gestão Empresarial - EaD'}]
#listaCursosFatec = []

for c in range (len(listaCursos)):
    key = int((divCursos.find_all('a')[c]['href']).split("=")[1])
    value= divCursos.find_all("a")[c].get_text()
    listaCursosFatec.append({'id':key,'curso':value})
#listaCursosFatec.appemd({'id':245,'curso':'Gestão Empresarial - EaD'})
dfCursos = pd.DataFrame(listaCursosFatec)

dfCursos.head()

,id,curso
0,245,Gestão Empresarial - EaD
1,247,Agroindústria
2,75,Agronegócio
3,76,Alimentos
4,266,Análise de Processos Agroindustriais


In [344]:
dfCursos.to_csv('ds/listaCursos.csv', sep=';',index=False)


In [345]:
classificacao_geral_vest_fatec = webdriver.Chrome()

classificacao_geral_vest_fatec.get("https://www.vestibularfatec.com.br/classificacao/fatec.asp")

In [347]:
selectFatec = Select(classificacao_geral_vest_fatec.find_element_by_id('CodFatec'))

select_id_fatecs = [fatec['value'] for fatec in fatecs]
select_value_fatecs = [fatec.string for fatec in fatecs]


In [348]:

for i in tqdm(range(len (dfFatec))):
    #print(f'ATUALIZANDO O REGISTRO {i}', end='\n\n')
    id_ = dfFatec.loc[i,'id']
    fatec_ = dfFatec.loc[i,'fatec']
    selectFatec.select_by_visible_text(fatec_)
    #CLICAR NO BOTAO ENTRAR
    classificacao_geral_vest_fatec.find_element_by_xpath('//*[@id="formClassificacao"]/div[2]/button').click()
    time.sleep(3)
    
    selectCursos = Select(classificacao_geral_vest_fatec.find_element_by_id('CodEscolaCurso'))
    print(fatec_, i)
    for c in range(len (selectCursos.options)):
        if 'Selecione...' not in selectCursos.options[c].text:
            print(selectCursos.options[c].text)
            selectCursos.select_by_visible_text(selectCursos.options[c].text)    
            classificacao_geral_vest_fatec.find_element_by_xpath('//*[@id="formClassificacao"]/div[2]/button').click()   
            time.sleep(2)   
            url_lista_1chamada = classificacao_geral_vest_fatec.current_url
            #url_lista_1chamada = 'https://www.vestibularfatec.com.br/classificacao/lista.asp'
            url_lista_1chamada = (f'{url_lista_1chamada}?codfatec={id_}&codescolacurso={2005}&o=1')
            classificacao_geral_vest_fatec.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/ul/li[1]/a').click()   

            #lista.asp?codfatec=1&codescolacurso=902&o=1
        '''if 'Selecione...' not in selectCursos.options[c].text:
            selectCursos.select_by_visible_text(row['curso'])    
            classificacao_geral_vest_fatec.find_element_by_xpath('//*[@id="formClassificacao"]/div[2]/button').click()   
            time.sleep(2)
        '''
    classificacao_geral_vest_fatec.back()


  0%|          | 0/73 [00:00<?, ?it/s]

Adamantina - Fatec Adamantina 0
Ciência de Dados (Noite)


  0%|          | 0/73 [00:08<?, ?it/s]


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=88.0.4324.150)


In [282]:
dfCursos.head()

,id,curso
0,245,Gestão Empresarial - EaD
1,247,Agroindústria
2,75,Agronegócio
3,76,Alimentos
4,266,Análise de Processos Agroindustriais


In [309]:
selectCursos = Select(classificacao_geral_vest_fatec.find_element_by_id('CodEscolaCurso'))

#selectCursos.options[2].text

#del selectCursos.options[0]


#selectCursos = selectCursos.options.pop(0)
len(selectCursos.options)

3

In [310]:
selectCursos = Select(classificacao_geral_vest_fatec.find_element_by_id('CodEscolaCurso'))

for c in range(len (selectCursos.options)):
    if 'Selecione...' not in selectCursos.options[c].text:
        selectCursos.select_by_visible_text(row['curso'])    
        classificacao_geral_vest_fatec.find_element_by_xpath('//*[@id="formClassificacao"]/div[2]/button').click()   
        time.sleep(2)

NoSuchElementException: Message: Could not locate element with visible text: Gestão Empresarial - EaD


In [328]:
selectCursos.options[1].value

    #print(fatec.string)
    #print(fatec.get_text())

AttributeError: 'WebElement' object has no attribute 'value'

In [ ]:
body = soup.find('body')
#print(body.prettify())
#body.p

In [208]:
classificacao_geral_vest_fatec.current_url

'https://www.vestibularfatec.com.br/classificacao/lista.asp'

In [316]:
selectCursos.options[0].

SyntaxError: invalid syntax (<ipython-input-316-a70d0e817268>, line 1)

In [212]:
divCursos.find_all('a')[0]['href']
#divCursos.find_all('a')[0].contents[0]
#divCursos.find_all("a")[2].get_text()
#len (listaCursos)

'curso.asp?c=247'

In [192]:
divCursos.find_all("a")


    print(div.find('a')['href'])
    print(div.find('a').contents[0])

IndentationError: unexpected indent (<ipython-input-192-4477cc19bad4>, line 4)

In [ ]:
https://www.vestibularfatec.com.br/classificacao/lista.asp?codfatec=2&codescolacurso=2005&o=1
    

In [335]:
url_lista_1chamada = 'https://www.vestibularfatec.com.br/classificacao/lista.asp'
url_lista_1chamada = (f'{url_lista_1chamada}?codfatec={123}&codescolacurso={2005}&o=1')
url_lista_1chamada
#lista.asp?codfatec=1&codescolacurso=902&o=1

'https://www.vestibularfatec.com.br/classificacao/lista.asp?codfatec=123&codescolacurso=2005&o=1'

In [ ]:
for i, row in dfFatec.iterrows():
    print(row['id'], row['fatec'])
    id_ = row['id']
    fatec_ = row['fatec']
    selectFatecs.select_by_visible_text(fatec_)
    #CLICAR NO BOTAO ENTRAR
    classificacao_geral_vest_fatec.find_element_by_xpath('//*[@id="formClassificacao"]/div[2]/button').click()
    time.sleep(5)

    #selectCursos = popularSelect(classificacao_geral_vest_fatec,'CodEscolaCurso')
    selectCursos = Select(classificacao_geral_vest_fatec.find_element_by_id('CodEscolaCurso'))

    time.sleep(2)
    print(f'quantidade de cursos {len(selectCursos.options)}')
    
    for c, row2 in enumerate(selectCursos.options):
    #for c in range(len (selectCursos.options)):
        if 'Selecione...' not in row2.text:
            print(f'{fatec_} - {row2.text}')
            
            selectCursos.select_by_visible_text(row2.text)
                                  
            classificacao_geral_vest_fatec.find_element_by_xpath('//*[@id="formClassificacao"]/div[2]/button').click()   
            
            #time.sleep(2)   
            #url_lista_1chamada = classificacao_geral_vest_fatec.current_url
            #cod_curso = int(row2.get_attribute("value"))
            #url_lista_1chamada = (f'{url_lista_1chamada}?codfatec={id_}&codescolacurso={cod_curso}&o=1')
            #classificacao_geral_vest_fatec.find_element_by_xpath('/html/body/div[2]/div/div[2]/div/ul/li[1]/a').click()   

            #content = classificacao_geral_vest_fatec.page_source
           # with open(f'{fatec_}_{selectCursos.options[c].text}.html', 'w') as f:
                #f.write(content)
               # time.sleep(1)
            time.sleep(2)
        classificacao_geral_vest_fatec.back()
        time.sleep(1)
    classificacao_geral_vest_fatec.back()
    time.sleep(2)

In [1]:
case='12'

In [4]:
case.isdigit()

False

In [3]:
case='abc'
